## Phân tích Sản phẩm Sâu
#### **Mục tiêu (Objectives)**

- Phân tích hiệu suất sản phẩm
- Xác định sản phẩm bán chạy và dẫn đầu doanh thu
- Phân tích tốc độ bán và tỷ lệ bán hết
- Đánh giá vòng đời sản phẩm


## 1. Import và setup


In [12]:
# Import các thư viện cần thiết
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)      # Hiển thị tất cả cột
pd.set_option('display.width', None)            # Không giới hạn chiều rộng
pd.set_option('display.max_colwidth', None)     # Không giới hạn độ rộng cột

# Load biến môi trường từ file .env
load_dotenv()

# Lấy thông tin kết nối database
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_GOLD = os.getenv("DB_GOLD")

# Tạo kết nối tới Gold database
gold_engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_GOLD}")


## 2. Load data


In [13]:
# Load du lieu tat ca cac bang

df_orders = pd.read_sql_table("fact_orders", gold_engine)
df_orders_items = pd.read_sql_table("fact_order_items", gold_engine)
df_products = pd.read_sql_table("dim_products", gold_engine)
df_customers = pd.read_sql_table("dim_customers", gold_engine)
df_date = pd.read_sql_table("dim_date", gold_engine)
df_pages = pd.read_sql_table("dim_order_pages", gold_engine)
df_payments = pd.read_sql_table("dim_order_payments", gold_engine)
df_shipping = pd.read_sql_table("dim_order_shipping", gold_engine)
df_warehouses = pd.read_sql_table("dim_order_warehouses", gold_engine)
df_shop = pd.read_sql_table("dim_shop", gold_engine)


## Nội dung phân tích (Analysis Content)


### A. Product Performance Metrics (Chỉ số hiệu suất sản phẩm)
Product Performance Metrics giúp đánh giá hiệu quả của từng sản phẩm:
- Best Sellers: Sản phẩm bán chạy nhất
- Revenue Leaders: Sản phẩm dẫn đầu doanh thu
- Product Velocity: Tốc độ bán hàng
- Sell-through Rate: Tỷ lệ bán hết hàng


#### 1. Best Sellers (Sản phẩm bán chạy)


In [15]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1386 entries, 0 to 1385
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date_id           1386 non-null   int64         
 1   full_date         1386 non-null   datetime64[ns]
 2   year              1386 non-null   int64         
 3   month             1386 non-null   int64         
 4   day               1386 non-null   int64         
 5   day_of_week       1386 non-null   int64         
 6   weekday_name      1386 non-null   object        
 7   day_of_year       1386 non-null   int64         
 8   week_of_year      1386 non-null   int64         
 9   quarter           1386 non-null   int64         
 10  quarter_name      1386 non-null   object        
 11  month_name        1386 non-null   object        
 12  year_month        1386 non-null   object        
 13  year_quarter      1386 non-null   object        
 14  is_weekend        1386 n

In [14]:
# Tính toán sản phẩm bán chạy
def analyze_best_sellers():
    """
    Phân tích sản phẩm bán chạy dựa trên số lượng bán được
    """
    
    # Merge orders_items với products để có thông tin sản phẩm
    order_items_with_products = df_orders_items.merge(df_products, on='product_id', how='inner')
    
    # Merge với orders để có thông tin thời gian
    order_items_with_products = order_items_with_products.merge(df_orders[['order_id', 'date_id']], on='order_id', how='left')
    order_items_with_products = order_items_with_products.merge(df_date, on='date_id', how='left')
    
    # Tính toán metrics theo sản phẩm
    best_sellers = order_items_with_products.groupby(['product_id', 'name']).agg({
        'order_item_id': 'count',              # Số lần được mua
        'item_quantity': 'sum',                # Tổng số lượng bán
        'item_price': 'sum',                   # Tổng doanh thu
        'customer_id': 'nunique',              # Số khách hàng unique
        'full_date': ['min', 'max']            # Ngày bán đầu tiên và cuối
    }).reset_index()
    
    # Flatten column names
    best_sellers.columns = ['product_id', 'product_name', 'times_purchased', 'total_quantity_sold', 
                            'total_revenue', 'unique_customers', 'first_sale_date', 'last_sale_date']
    
    # Tính số ngày hoạt động
    best_sellers['days_active'] = (best_sellers['last_sale_date'] - best_sellers['first_sale_date']).dt.days + 1
    
    # Sắp xếp theo số lượng bán
    best_sellers = best_sellers.sort_values('total_quantity_sold', ascending=False)
    
    return best_sellers

# Chạy phân tích
best_sellers = analyze_best_sellers()
print("=== BEST SELLERS (SẢN PHẨM BÁN CHẠY) ===")
print(f"Phân tích {len(best_sellers)} sản phẩm")
print(f"Tổng số lượng bán: {best_sellers['total_quantity_sold'].sum():,}")
print("-" * 80)
best_sellers.head(15)


ValueError: You are trying to merge on object and int64 columns for key 'date_id'. If you wish to proceed you should use pd.concat

#### 2. Revenue Leaders (Sản phẩm dẫn đầu doanh thu)


In [ ]:
# Tính toán sản phẩm dẫn đầu doanh thu
def analyze_revenue_leaders():
    """
    Phân tích sản phẩm dẫn đầu doanh thu
    """
    
    # Sử dụng kết quả từ best_sellers và sắp xếp lại theo doanh thu
    revenue_leaders = best_sellers.copy()
    revenue_leaders = revenue_leaders.sort_values('total_revenue', ascending=False)
    
    # Tính thêm metrics
    revenue_leaders['avg_revenue_per_sale'] = revenue_leaders['total_revenue'] / revenue_leaders['times_purchased']
    revenue_leaders['avg_revenue_per_customer'] = revenue_leaders['total_revenue'] / revenue_leaders['unique_customers']
    
    return revenue_leaders

# Chạy phân tích
revenue_leaders = analyze_revenue_leaders()
print("=== REVENUE LEADERS (SẢN PHẨM DẪN ĐẦU DOANH THU) ===")
print(f"Tổng doanh thu: {revenue_leaders['total_revenue'].sum():,.0f} VND")
print("-" * 80)
revenue_leaders.head(15)


#### 3. Product Velocity (Tốc độ bán)


In [ ]:
# Tính toán tốc độ bán của sản phẩm
def analyze_product_velocity():
    """
    Phân tích tốc độ bán sản phẩm
    Product Velocity = total_quantity_sold / days_active
    """
    
    # Sử dụng kết quả từ best_sellers
    product_velocity = best_sellers.copy()
    
    # Tính product velocity
    product_velocity['product_velocity'] = product_velocity['total_quantity_sold'] / product_velocity['days_active']
    
    # Sắp xếp theo velocity
    product_velocity = product_velocity.sort_values('product_velocity', ascending=False)
    
    return product_velocity

# Chạy phân tích
product_velocity = analyze_product_velocity()
print("=== PRODUCT VELOCITY (TỐC ĐỘ BÁN) ===")
print(f"Trung bình tốc độ bán: {product_velocity['product_velocity'].mean():.2f} sản phẩm/ngày")
print("-" * 80)
product_velocity.head(15)


#### 4. Sell-through Rate (Tỷ lệ bán hết)


In [ ]:
# Tính toán tỷ lệ bán hết sản phẩm
def analyze_sell_through_rate():
    """
    Phân tích tỷ lệ bán hết sản phẩm
    Sell-through Rate = quantity_sold / (quantity_sold + remain_quantity) * 100
    """
    
    # Merge products với best_sellers để có thông tin tồn kho
    product_metrics = best_sellers.merge(df_products[['product_id', 'total_quantity', 'remain_quantity']], 
                                        on='product_id', how='left')
    
    # Tính sell-through rate
    product_metrics['sell_through_rate'] = (product_metrics['total_quantity_sold'] / 
                                            (product_metrics['total_quantity_sold'] + product_metrics['remain_quantity']) * 100)
    product_metrics['sell_through_rate'] = product_metrics['sell_through_rate'].fillna(0)
    
    # Tính inventory turnover
    product_metrics['inventory_turnover'] = product_metrics['total_quantity_sold'] / product_metrics['days_active']
    
    # Sắp xếp theo sell-through rate
    product_metrics = product_metrics.sort_values('sell_through_rate', ascending=False)
    
    return product_metrics

# Chạy phân tích
product_metrics = analyze_sell_through_rate()
print("=== SELL-THROUGH RATE (TỶ LỆ BÁN HẾT) ===")
print(f"Trung bình tỷ lệ bán hết: {product_metrics['sell_through_rate'].mean():.2f}%")
print("-" * 80)
product_metrics.head(15)


#### 5. Product Lifecycle Analysis (Phân tích vòng đời sản phẩm)


In [ ]:
# Phân tích vòng đời sản phẩm theo thời gian
def analyze_product_lifecycle():
    """
    Phân tích vòng đời sản phẩm và hiệu suất theo thời gian
    """
    
    # Merge orders_items với products và date
    order_items_full = df_orders_items.merge(df_products[['product_id', 'name', 'inserted_at']], 
                                            on='product_id', how='left')
    order_items_full = order_items_full.merge(df_orders[['order_id', 'date_id']], on='order_id', how='left')
    order_items_full = order_items_full.merge(df_date, on='date_id', how='left')
    
    # Tính lifecycle stage cho mỗi sản phẩm
    def classify_lifecycle_stage(row):
        """
        Phân loại giai đoạn vòng đời sản phẩm
        """
        days_active = row['days_active']
        velocity = row['product_velocity']
        sell_through = row['sell_through_rate']
        
        if days_active <= 30:
            return 'Product Mới'
        elif velocity > product_velocity['product_velocity'].quantile(0.75) and sell_through > 50:
            return 'Product Đang Tăng Trưởng'
        elif velocity > product_velocity['product_velocity'].quantile(0.5):
            return 'Product Ổn Định'
        elif velocity > product_velocity['product_velocity'].quantile(0.25):
            return 'Product Suy Giảm'
        else:
            return 'Product Cần Cải Thiện'
    
    # Merge với product metrics
    lifecycle_analysis = product_metrics.copy()
    lifecycle_analysis['lifecycle_stage'] = lifecycle_analysis.apply(classify_lifecycle_stage, axis=1)
    
    return lifecycle_analysis

# Chạy phân tích
lifecycle_analysis = analyze_product_lifecycle()
print("=== PRODUCT LIFECYCLE ANALYSIS (PHÂN TÍCH VÒNG ĐỜI SẢN PHẨM) ===")
print()
print("Phân bố theo giai đoạn vòng đời:")
stage_counts = lifecycle_analysis['lifecycle_stage'].value_counts()
for stage, count in stage_counts.items():
    print(f"   {stage}: {count} sản phẩm")
print("-" * 80)
lifecycle_analysis.head(20)
